In [1]:
from TNT_Api import TNT_Api
UN = TNT_Api(tag='聯合國',nlimit=100000)

In [3]:
UN.get_data()

1377 documents found in tag 聯合國
downloading, 100000 to go
0 data downloaded
100 data downloaded
200 data downloaded
300 data downloaded
400 data downloaded
500 data downloaded
600 data downloaded
700 data downloaded
800 data downloaded
900 data downloaded
1000 data downloaded
1100 data downloaded
1200 data downloaded
1300 data downloaded


In [15]:
## Simple Cleaning and Spelling Correction
ocr = UN.ocr.copy().reset_index()
def replace_line_change(t):
    if len(t)>0: return t[0].replace('\n',' ')
    else: return None
ocr['ocr_text'] = ocr.ocr.map(replace_line_change,na_action='ignore')

In [29]:
## Remove non-English
from langdetect import detect, DetectorFactory
from langdetect import detect_langs
DetectorFactory.seed =0 
def remove_non_eng(text):
    if detect(text)=='en':return text
    else: return None
ocr['ocr_eng'] = ocr.ocr_text.map(remove_non_eng,na_action='ignore')

In [31]:
(ocr.ocr_eng.isnull()).sum()

93

In [35]:
ocr = ocr.dropna(axis=0,how='any')

In [32]:
import enchant
import spelling_correction as splng_cr
chkr = enchant.Dict('en_US')

In [ ]:
## test cell
import spelling_correction as sc

In [49]:
import numpy as np
test = ocr.ocr_eng.copy()
def check_mis_spell(text):
    mis_rate = np.sum([chkr.check(w) for w in sc.words(text)])/len(sc.words(text))
    return mis_rate
mis_rate = test.map(check_mis_spell,na_action='ignore')

In [51]:
mis_rate.describe()

count    1284.000000
mean        0.744819
std         0.129981
min         0.142857
25%         0.647444
50%         0.763569
75%         0.849251
max         1.000000
Name: ocr_eng, dtype: float64

In [52]:
text_length = test.map(len,na_action='ignore')

In [54]:
np.sum([text_length>100])

1262

In [56]:
total_words = text_length.sum()
total_words

2199153

In [58]:
def auto_correct(text):
    wordlist = list()
    for w in sc.words(text):
        if chkr.check(w): wordlist.append(w)
        else: wordlist.append(sc.correction(w))
    art = " ".join(wordlist)
    return art
import time
start = time.time()
subset = test.iloc[:100].map(auto_correct,na_action='ignore')
end = time.time()-start
print (end)

225.57488632202148


In [62]:
subset.head()

0    activities for regional nurses the regional nu...
1    to hug lien key map legend chingshu existing p...
2    the office of ohio t laepeetor serene that dor...
4    b ground water development in the muslin and c...
5    ten page 10 sodium bromide phonoborbital ferri...
Name: ocr_eng, dtype: object

In [81]:
from string import digits
remove_digits = str.maketrans('','',digits)

def remove_number(text):
    return text.translate(remove_digits)
subset = subset.map(remove_number,na_action='ignore')

In [88]:
subset.to_csv('data/UN_subset_checkpoint.csv',index=False,header=True)

### Check point

In [17]:
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import bokeh.plotting as bp
from bokeh.plotting import save
from bokeh.models import HoverTool
UN_full_text = pd.read_csv('data/UN_full_text.csv')
UN_full_text.head()

,ocr_eng
0,activities for regional nurses the regional nu...
1,to hug lien key map legend chingshu existing p...
2,the office of ohio t laepeetor serene that dor...
3,b ground water development in the muslin and c...
4,ten page 10 sodium bromide phonoborbital ferri...


In [14]:
from string import digits
remove_digits = str.maketrans('','',digits)

def remove_number(text):
    return text.translate(remove_digits)
UN_full_text = UN_full_text.ocr_eng.map(remove_number,na_action='ignore')

In [23]:
documents = list(UN_full_text.ocr_eng)

In [25]:
no_features = 100

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

no_topics = 10
# Run NMF
NMF = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
# Run LDA
LDA = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]\
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
print ("========Non-Negative Matrix Factorization===========")
display_topics(NMF, tfidf_feature_names, no_top_words)
print ("========Latent Dirichlet Allocation=================")
display_topics(LDA, tf_feature_names, no_top_words)

/Users/yhhsu/Anaconda/anaconda/envs/NLP/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


========Non-Negative Matrix Factorization===========
Topic 0:
carry intra supplies office regional personnel china program relief director
Topic 1:
river sediment flood stations basin data site areas later bed
Topic 2:
hospital nurses nursing medical beds health red 10 shanghai cases
Topic 3:
dr medical health supplies division meeting report action shanghai present
Topic 4:
000 50 tons 10 20 total 100 12 people years
Topic 5:
shop branch national central new meeting 50 total plan equipment
Topic 6:
grade officer regional 12 division chief 11 10 100 field
Topic 7:
tho th end tons government people distribution time island control
Topic 8:
mr meeting director china committee office intra chief staff plan
Topic 9:
hydraulic laboratory equipment work development problems planning taiwan available government
========Latent Dirichlet Allocation=================
Topic 0:
stations 100 used taiwan tons island period years general new
Topic 1:
river sediment flood bed control data areas present